Link consultado: https://cloud.google.com/bigquery/docs/samples/bigquery-pandas-gbq-to-gbq-simple?hl=pt-br


Importando as Bibliotecas

In [3]:
from google.cloud import bigquery   
from google.oauth2 import service_account 
import pandas as pd
import requests 
import json
import numpy as np
import pandas_gbq

Criando o cliente de conexão com o Bigquery. Você precisa subir a sua service account.

In [6]:
credencial = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/SIRIUS/Dados/Chaves/gabriel-sirius-e7b998baf018.json')

client = bigquery.Client(credentials=credencial, project=credencial.project_id)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Selecionando dados do Bigquery

In [7]:
df_municipios = client.query('''SELECT * FROM `basedosdados.br_ibge_populacao.municipio` where ano = 2021 and populacao > 100000''').to_dataframe()

In [8]:
df_municipios.head()

,ano,sigla_uf,id_municipio,populacao
0,2021,RO,1100023,111148
1,2021,RO,1100122,131026
2,2021,RO,1100205,548952
3,2021,RO,1100304,104517
4,2021,AC,1200401,419452


Preparando para consultar a API. Aqui você precisa da sua chave de API.

In [9]:
chave = '819a85baff3f54451c8f2ce5f92bfddf'

chave_dict = {'chave-api-dados': chave}

Iterando cada municipio para realizar a chamada na API.

In [10]:
list_ids = df_municipios.id_municipio.tolist()

serie_final = []

for id in list_ids:
  url = f'https://api.portaldatransparencia.gov.br/api-de-dados/bolsa-familia-por-municipio?codigoIbge={id}&mesAno=202111&pagina=1'
  result = requests.get(url, verify=True, headers=chave_dict).json()
  serie_final += result

In [11]:
df_final = pd.json_normalize(serie_final)
df_final.head(10)

,id,dataReferencia,valor,quantidadeBeneficiados,municipio.codigoIBGE,municipio.nomeIBGE,municipio.codigoRegiao,municipio.nomeRegiao,municipio.pais,municipio.uf.sigla,municipio.uf.nome,tipo.id,tipo.descricao,tipo.descricaoDetalhada
0,201902817,2021-11-01,841397.0,4403,1100023,ARIQUEMES,1,NORTE,BRASIL,RO,RONDÔNIA,1,Bolsa Família,Bolsa Família
1,207702394,2021-11-01,475821.0,2505,1100122,JI-PARANÁ,1,NORTE,BRASIL,RO,RONDÔNIA,1,Bolsa Família,Bolsa Família
2,187901825,2021-11-01,5056739.0,25485,1100205,PORTO VELHO,1,NORTE,BRASIL,RO,RONDÔNIA,1,Bolsa Família,Bolsa Família
3,212903389,2021-11-01,517514.0,2701,1100304,VILHENA,1,NORTE,BRASIL,RO,RONDÔNIA,1,Bolsa Família,Bolsa Família
4,210202149,2021-11-01,6976427.0,25773,1200401,RIO BRANCO,1,NORTE,BRASIL,AC,ACRE,1,Bolsa Família,Bolsa Família
5,189503054,2021-11-01,3081938.0,10659,1301902,ITACOATIARA,1,NORTE,BRASIL,AM,AMAZONAS,1,Bolsa Família,Bolsa Família
6,215403372,2021-11-01,25732813.0,129637,1302603,MANAUS,1,NORTE,BRASIL,AM,AMAZONAS,1,Bolsa Família,Bolsa Família
7,198902676,2021-11-01,4166383.0,15368,1303403,PARINTINS,1,NORTE,BRASIL,AM,AMAZONAS,1,Bolsa Família,Bolsa Família
8,210201831,2021-11-01,4461832.0,22085,1400100,BOA VISTA,1,NORTE,BRASIL,RR,RORAIMA,1,Bolsa Família,Bolsa Família
9,212202455,2021-11-01,7956344.0,32987,1500107,ABAETETUBA,1,NORTE,BRASIL,PA,PARÁ,1,Bolsa Família,Bolsa Família


In [12]:
df_final.rename(columns={'municipio.codigoIBGE': 'id_municipio',
                         'quantidadeBeneficiados':'quantidadeBeneficiadosBolsaFamilia',
                         'valor':'valorBolsaFamilia',
                         'dataReferencia':'dataReferenciaBolsaFamilia'},inplace=True)

df_final_bolsa = df_final[['dataReferenciaBolsaFamilia','valorBolsaFamilia', 'quantidadeBeneficiadosBolsaFamilia','id_municipio']]

df_pop_bolsa = df_municipios.merge(df_final_bolsa, how='inner', on='id_municipio')

df_pop_bolsa.head()

,ano,sigla_uf,id_municipio,populacao,dataReferenciaBolsaFamilia,valorBolsaFamilia,quantidadeBeneficiadosBolsaFamilia
0,2021,RO,1100023,111148,2021-11-01,841397.0,4403
1,2021,RO,1100122,131026,2021-11-01,475821.0,2505
2,2021,RO,1100205,548952,2021-11-01,5056739.0,25485
3,2021,RO,1100304,104517,2021-11-01,517514.0,2701
4,2021,AC,1200401,419452,2021-11-01,6976427.0,25773


Salvando os dados no bigquery.

In [2]:
# table_id = "gabriel-sirius.Municipios.tabela_municipio"

# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",
# )

# job = client.load_table_from_dataframe(
#     df_pop_bolsa, table_id, job_config=job_config
# )  
# job.result()  

# table = client.get_table(table_id)

# print(
#     "Loaded {} rows and {} columns to {}".format(
#         table.num_rows, len(table.schema), table_id
#     )
# )

In [17]:
project_id='gabriel-sirius'
table_id='my_dataset.municipio'
pandas_gbq.to_gbq(df_pop_bolsa,table_id,project_id=project_id)

1it [00:05,  5.82s/it]
